# The Single Neuron

The Single Neuron project.

Dataset: https://www.kaggle.com/dansbecker/hot-dog-not-hot-dog
Inspirations:
  - https://towardsdatascience.com/a-logistic-regression-from-scratch-3824468b1f88


In [ ]:
import numpy as np